In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import DataAgent, TacProcess
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *


In [2]:
#data_gen = DataGenerator(random=False, base_dir = "data/4jul21/")

data_gen = DataGenerator(random=False, base_dir = "data/backtest/")

#data_gen = DataGenerator(random=False, base_dir = "data/btcusd/")

data_gen.rewind()


In [3]:
from joblib import dump, load

models = [
    #load('model/StackingClassifier'), 
    #load('model/RandomForestClassifier_accuracy_score'),
    load('model/RandomForestClassifier_f1_score'),
    load('model/RandomForestClassifier_precision_score'),
    #load('model/QuadraticDiscriminantAnalysis'),
    #load('model/RandomForestClassifier2')
]

print(f"{len(models)}")

2


In [4]:
x, y = load_data("backtest", "train")

model = models[0]

eval_data(model, x, y)

precision_score(y, model.predict(x))

              precision    recall  f1-score   support

     class 0       0.58      0.40      0.47       985
     class 1       0.48      0.66      0.56       840

    accuracy                           0.52      1825
   macro avg       0.53      0.53      0.51      1825
weighted avg       0.53      0.52      0.51      1825



0.4821583986074848

In [5]:
data_count = data_gen.steps
data_count = 200000 #data_gen.steps
data_gen.rewind()

raw_data = []

for i in tqdm(range(data_count)):
    raw_data.append(data_gen.next())

<ipython-input-5-a9d8ccaca572>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/200000 [00:00<?, ?it/s]

In [6]:
def process_model(model, resample):
    back = BackTest(value = 100)

    request_sell = lambda price: back.request_sell(price)
    request_buy = lambda price: back.request_buy(price)
    on_state = lambda timestamp, price: back.on_state(timestamp, price)

    stock = StockAgent(model = model,
                    request_sell = request_sell,
                    request_buy = request_buy,
                    on_state = on_state
                      )


    data_gen.rewind()
    back.reset()

    on_new_data = lambda x: print(x)
    on_new_data = lambda x: stock.on_x(x)

    on_state = lambda timestamp, price, bid, ask: print("{} {} {} {}".format(timestamp, price, bid[-1], ask[-1]))
    on_state = lambda timestamp, price, bid, ask: stock.on_new_state(timestamp, price, bid, ask)


    agent = DataAgent(
        resample = resample,
        on_state = on_state,
        on_new_data = on_new_data
    )

    on_new_data_count = 0

    for raw in tqdm(raw_data):
        agent.on_new_raw_data(raw)

    if (back.bought):
        stock.sell()

    return back

In [7]:
scores = []
for idx, model in enumerate(models):
    back = process_model(model, '2Min')
    scores.append((idx, back))
    

Resample 2Min - TecAn ( windows 20, windows_limit 100 )


<ipython-input-6-bcd02c62a8b5>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for raw in tqdm(raw_data):


  0%|          | 0/200000 [00:00<?, ?it/s]

2021-06-25 21:12:06 Buy (31755.8): ask: 31755.8
2021-06-25 21:34:08 Sell (31895.83) 140.03000000000247 total: 100.44095881697203
2021-06-25 21:50:20 Buy (31803.26): ask: 31803.26
2021-06-25 22:12:02 Sell (31901.61) 98.35000000000218 total: 100.75020384405416
2021-06-25 22:14:01 Buy (31910.81): ask: 31910.81
2021-06-25 22:26:06 Sell (31895.32) -15.4900000000016 total: 100.7016622996684
2021-06-25 22:28:13 Buy (31860.56): ask: 31860.56
2021-06-25 23:32:05 Sell (31590.0) -270.5600000000013 total: 99.8524619089659
2021-06-25 23:48:26 Buy (31807.59): ask: 31807.59
2021-06-25 23:54:05 Sell (31744.91) -62.68000000000029 total: 99.65540202483133
2021-06-25 23:56:16 Buy (31664.36): ask: 31664.36
2021-06-26 00:08:10 Sell (31748.18) 83.81999999999971 total: 99.92011604399988
2021-06-26 00:10:02 Buy (31485.22): ask: 31485.22
2021-06-26 00:20:07 Sell (31828.16) 342.9399999999987 total: 101.00932552070037
2021-06-26 00:24:04 Buy (31772.37): ask: 31772.37
2021-06-26 00:30:02 Sell (31873.38) 101.01000

<ipython-input-6-bcd02c62a8b5>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for raw in tqdm(raw_data):


  0%|          | 0/200000 [00:00<?, ?it/s]

2021-06-25 21:16:07 Buy (31602.21): ask: 31602.21
2021-06-25 21:18:02 Sell (31580.51) -21.700000000000728 total: 99.93133391620397
2021-06-25 22:34:16 Buy (31761.05): ask: 31761.05
2021-06-25 22:36:13 Sell (31725.63) -35.419999999998254 total: 99.81981367364273
2021-06-26 01:18:01 Buy (31823.9): ask: 31823.9
2021-06-26 01:24:01 Sell (31717.59) -106.31000000000131 total: 99.48575656562014
2021-06-26 01:34:08 Buy (31788.43): ask: 31788.43
2021-06-26 01:42:07 Sell (31650.17) -138.26000000000204 total: 99.05081844505237
2021-06-26 01:46:21 Buy (31537.54): ask: 31537.54
2021-06-26 01:48:25 Sell (31695.71) 158.16999999999825 total: 99.55234773363988


In [8]:
for idx, back in scores:
    print(f'{idx} {back.current}')

0 102.51040207337985
1 99.55234773363988
